# Data 620 - Final Project

Baron Curtin, Heather Geiger

RNA sequencing has been a valuable tool in genetics research for well over a decade now. By looking at RNA, one can look at which genes are expressed in addition to just looking at genetic variation.

More recently, RNA sequencing at the single cell level rather than just of the whole tissue lets you see which genes are expressed in subsets of a tissue or particular cell types. Popular applications include development, neurology, autoimmune disease, and oncology.

In our final project, we plan to use automated data analysis to get a broad overview of the published research on single-cell RNA sequencing. Specifically, we plan to focus on two aspects of this. 

The first aspect will be to get a sense of the most influential authors in this field, and the relationships between them. To do this, we will run a network analysis, where connections are defined by authors being on the same paper together. 

The second aspect will be to get a sense of recurrent themes in the literature through text mining. Here, we will focus on recurrent terms in the abstracts to do this. Since the abstract of a paper is meant to provide a summary of the most important themes in a single paragraph or two, text mining these should provide a lot of information despite only needing to process a relatively small amount of text.

In [36]:
# standard imports
import pickle
import gzip
from pathlib import Path

# third party imports
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer

from Bio import Entrez
from Bio import Medline
from tqdm import tqdm

# stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# additional jupyter setup
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


ModuleNotFoundError: No module named 'cPickle'

## Load Data

Lets load our data and perform some EDA to get a feel for our data.

In [50]:
# load data
data_paths = (Path.cwd() / 'data').glob('*.gz')

def parse_data(data_path):
    with gzip.open(data_path, 'rb') as pf:
        loaded_data = pickle.load(pf)
    return loaded_data

data = [x for dp in data_paths for x in parse_data(dp)]
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

